In [187]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [188]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'USE_CASE', 'INCOME_AMT'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,5000,1
1,T3,Independent,C2000,Co-operative,108590,1
2,T5,CompanySponsored,C3000,Association,5000,0
3,T3,CompanySponsored,C2000,Trust,6692,1
4,T3,Independent,C1000,Trust,142590,1
...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,5000,0
34295,T4,CompanySponsored,C3000,Association,5000,0
34296,T3,CompanySponsored,C2000,Association,5000,0
34297,T5,Independent,C3000,Association,5000,1


In [189]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [190]:
application_df['ASK_AMT'].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [191]:
application_df['ASK_AMT'].min()

5000

In [192]:
application_df['ASK_AMT'].max()

8597806340

In [193]:
application_df['ASK_AMT'].mean()

2769198.6820607013

In [194]:
application_df['ASK_AMT'].median()

5000.0

In [195]:
ask_amts = [0, 10000, 100000, 1000000, 9000000000]
labels = ["<$10,000", "$10,000-100,000", "$100,000-1,000,000", "$1,000,000-$9,000,000,000"]

In [196]:
application_df["Ask_Amt_Ranges"] = pd.cut(application_df["ASK_AMT"], ask_amts, labels = labels, right=False)

In [197]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [198]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T17", "T15", "T29", "T14", "T25", "T2", "T12", "T13"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [199]:
# Look at CLASSIFICATION value counts for binning
value_counts = application_df['CLASSIFICATION'].value_counts()
value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [200]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df[application_df['CLASSIFICATION'].isin(value_counts.index[value_counts > 1])]
classification_counts = classification_counts['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [201]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ["C2561", "C1580", "C1900", "C1820", "C6100", "C1248", "C2600", "C1370", "C5200", "C4200", "C2190", "C1236", "C2570", "C2500", "C3700", "C1570", "C1283", "C2380", "C1732",
                              "C1728", "C2170", "C4120", "C8210", "C4500", "C1245", "C2150", "C3200", "C1246", "C1234", "C1256", "C1267", "C0", "C2710", "C1260", "C1257", "C1600", "C2400", "C1720", "C4100",
                              "C7210", "C1235", "C1237", "C1278", "C1238", "C8200", "C1250", "C6000", "C1800", "C1500", "C7120", "C8000", "C1240", "C2300", "C7200", "C1400", "C1230"]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [202]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'AFFILIATION','Ask_Amt_Ranges'])
application_df = pd.concat([application_df, dummy_df], axis=1)
application_df = application_df.drop(['APPLICATION_TYPE', 'CLASSIFICATION', 'ORGANIZATION', 'AFFILIATION', 'Ask_Amt_Ranges'], axis=1)

application_df

,ASK_AMT,IS_SUCCESSFUL,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,"Ask_Amt_Ranges_<$10,000","Ask_Amt_Ranges_$10,000-100,000","Ask_Amt_Ranges_$100,000-1,000,000","Ask_Amt_Ranges_$1,000,000-$9,000,000,000"
0,5000,1,5000,1,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,108590,1,108590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,5000,0,5000,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
3,6692,1,6692,1,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
4,142590,1,142590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,5000,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
34295,5000,0,5000,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
34296,5000,0,5000,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
34297,5000,1,5000,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0


In [203]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y)

In [204]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [205]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer_1 = 16
hidden_nodes_layer_2 = 10



nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer_1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer_2, activation="relu"))




# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 16)                688       
                                                                 
 dense_48 (Dense)            (None, 10)                170       
                                                                 
 dense_49 (Dense)            (None, 1)                 11        
                                                                 
Total params: 869 (3.39 KB)
Trainable params: 869 (3.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [206]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [207]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 1.8728 - accuracy: 0.6147
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 1.0642 - accuracy: 0.6800
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 1.1392 - accuracy: 0.6896
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.9006 - accuracy: 0.6828
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.8355 - accuracy: 0.7099
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.8075 - accuracy: 0.7112
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.7552 - accuracy: 0.7152
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.7377 - accuracy: 0.7166
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.7292 - accuracy: 0.7202
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.7260 - accura

In [208]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6170 - accuracy: 0.7301 - 457ms/epoch - 2ms/step
Loss: 0.616998553276062, Accuracy: 0.7301457524299622


In [209]:
nn.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
